# Application of Amplitude Estimation to Finance: Classical Finance

In this notebook, we will explore various concepts related to the estimation of *fair* option prices. Additionally, the following modules will be reviewed:

- **classical_finance**: This module implements several functions for computing payoffs for different financial derivatives and probability distributions.
- **probability_class**: This module includes the implementation of the **DensityProbability** Python class, which allows for the creation of probability distributions required for pricing derivative options.
- **payoff_class**: This module implements the **PayOff** Python class, which computes the payoff for various types of derivative options.

All these modules are part of the *finance* package within *QQuantLib* (**QQuantLib/finance**).

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## 1. The Problem

We have a **financial asset** whose value at time $ t $ is $ S(t) $. The **volatility** of the asset is $ \sigma $, and the **risk-free rate** is $ r $. Under these assumptions, we will create a **derivative contract** based on the evolution of the underlying asset with a fixed duration, known as the **maturity**, $ T $. This product will be characterized by its **return**, which is a function dependent on the price of the underlying at $ T $: $ f = f(S_T) $, where $ S_T = S(t=T) $. 

The key question is: **What is the 'fair' price of our derivative contract at time $ t $ when the underlying is $ S(t) $: $ V_f(S(t), t) $?**

Thus, we have:
- $ S = S(t) $: Price of the underlying at $ t $.
- $ \sigma $: Volatility of the underlying.
- **Risk-free rate**: $ r $.
- Return of the financial product: $ f = f(S_T) $.

And we aim to determine:
- $ V_f(S(t), t) $: The fair price of the derivative contract.

## 2. Black-Scholes Model

One model for obtaining $ V_f(S(t), t) $ is the **Black-Scholes** model, which relies on the following assumptions:

1. The underlying $ S(t) $ follows a log-normal random walk.
2. The risk-free interest rate $ r $ is a known function of time $ t $.
3. There are no dividends on the underlying $ S $.
4. Dynamic **delta hedging** (a risk elimination strategy) is applied.
5. No arbitrage opportunities exist.

Under these conditions, the **Black-Scholes** model describes the evolution of the asset $ S $ using the following **Stochastic Differential Equation (SDE)**:
$$
dS = \mu S dt + \sigma S dW
$$
where $ dW $ represents a Wiener process:
$$
dW \sim \mathcal{N}(0, \delta t), \quad \mathbf{E}[dW] = 0, \quad \mathbf{E}[dW^2] = dt.
$$

To establish a relationship between the underlying $ S $ and the price of the financial product $ V $, two approaches can be followed:

1. **Black-Scholes Partial Differential Equation (PDE):**
   $$
   \frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - rV = 0
   $$

2. **Expectation Computation Approach** (based on the Girsanov theorem):
   $$
   V(t, S(t)) = e^{-r(T-t)} \mathbb{E}[f(S_T \mid \mathcal{F}_t)]
   $$
   where $ \mathcal{F}_t $ represents the market information available up to time $ t $.

## 3. Price of Derivative Contract

So far, we have:
1. A financial asset with value $ S(t) $, volatility $ \sigma $, and risk-free rate $ r $.
2. A **derivative contract** based on the financial asset $ S $ with maturity $ T $ and a return function $ f = f(S(t), T) $.

In the *expectation approach*, the desired *fair* price of the option is given by:
$$
V(t, S(t)) = e^{-r(T-t)} \mathbb{E}[f(S_T \mid \mathcal{F}_t)]
$$

In general, to solve this problem, numerous **Monte Carlo** simulations of the evolution of the asset $ S $ between time $ t $ and the *maturity* $ T $ are performed under a specific financial model (such as **Black-Scholes** or **Heston**). The primary goal is to obtain the probability distribution of the asset at the maturity time:
$$
p(S_T; t, S(t))
$$

Once this function is obtained, it can be plugged into the expectation formula, yielding the standard method for calculating the *fair* price of options:
$$
V(t, S(t)) = e^{-r(T-t)} \mathbb{E}[f(S_T \mid \mathcal{F}_t)] = \int_0^\infty p(S_T; t, S(t)) f(S_T) dS_T
$$

## 4. Black-Scholes Probability Distribution

In the case of the pure **Black-Scholes** model, this probability distribution has an analytical form (the **Black-Scholes** probability distribution):

$$
P_{BS}(S_T; t, S(t)) = \frac{1}{S_T \sigma \sqrt{2\pi(T-t)}} \exp\left(-\frac{(\log S_T - \mu)^2}{2\sigma^2(T-t)}\right)
$$

where

$$
\mu = \left(r - \frac{1}{2}\sigma^2\right)(T-t) + \log S(t)
$$

Thus, the price of the financial product at time $ t $ and $ S(t) $ is given by:

$$
V(t, S(t)) = e^{-r(T-t)} \mathbb{E}_{P_{BS}}[f] = e^{-r(T-t)} \int_0^\infty P_{BS}(S_T; t, S(t)) f(S_T) dS_T
$$

The *return* of the financial product at the maturity time is the **Payoff** of the product: $ f(S_T) = \text{Payoff}(S_T) $.

This **Black-Scholes** probability density function is implemented in the `bs_probability` function from the `classical_finance` module (**QQuantLib/finance/classical_finance.py**).

In [ ]:
from QQuantLib.finance.classical_finance import bs_probability

In [ ]:
#Parameters of out model
S0 = 2
r = 0.04
sigma = 0.1
#maturity time for the derivative contract
T = 300/365 #300 days

In [ ]:
n_qbits = 5
x = np.linspace(1,3,2**n_qbits)
prob_bs = bs_probability(s_t=x, s_0=S0,risk_free_rate=r,volatility=sigma,maturity=T)

In [ ]:
#Black-Scholes Probability density
plt.plot(x, prob_bs)
plt.xlabel(r'$S_T$')
plt.ylabel(r'$P(S_T, S_0)$')
plt.title(r'Black-Scholes Probability density. r= {} $\sigma$= {}. $S_0$= {}'.format(r, sigma, S0))

So in the before graph we plot the probability of the value of the underlying S at maturity time T, following the **Black Scholes** model when risk free rate is $r$ and volatility of S is $\sigma$ and the initial value of the underlying is $S_0$.


### 4.1 DensityProbability Class

The `DensityProbability` class is a Python implementation designed to easily obtain the **Black-Scholes** probability distribution (with the intention of adding other financial probability distributions in the future). It is implemented in the *probability_class* module within the **finance** package.

The only mandatory input for creating this class is:

- `probability_type`: A string specifying the type of probability density to load (e.g., "Black-Scholes"). More probability distributions are expected to be added in the future.

The parameters required for the probability density should be provided as a dictionary. These parameters must align with the definition of the desired probability density function. The most common parameters include:

- `s_0`: Initial value of the asset ($ S(t) $).
- `risk_free_rate`: The risk-free rate.
- `volatility`: Volatility of the asset.
- `maturity`: The time at which the probability distribution over the asset's value is desired.

The main attribute of the class is `probability`, which represents the desired probability density function with the provided parameters fixed.

In general, the primary use of this class is to define the behavior of an asset and obtain its probability distribution at a specified *maturity* time.

In [ ]:
from QQuantLib.finance.probability_class import DensityProbability

In [ ]:
#Configuration  of a probability density
probability_type = "Black-Scholes"

density_dict = {
    "s_0": 2.0,
    "risk_free_rate": 0.05,
    "maturity": 0.5,
    "volatility": 0.5    
}

In [ ]:
bs_pdf = DensityProbability(probability_type, **density_dict)

As can be seen the *probability* property of the class is a function (in fact is a python partial function of the probability density desired).

In [ ]:
type(bs_pdf.probability)

In [ ]:
x = np.linspace(0.1, 6.0, 2**9)
plt.plot(x, bs_pdf.probability(x))

In [ ]:
#Playing with Black-Scholes

list_of_functions = []
#Lista = [1.0, 2.0, 3.0] #for s_0
#Lista = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2] #for maturity
#Lista = [0.01, 0.02, 0.03, 0.04, 0.05] #for risk_free_rate
Lista = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5] #for volatility

for i in Lista:
    step_dict = density_dict
    #step_dict.update({"s_0": i})
    #step_dict.update({"maturity": i})
    step_dict.update({"volatility": i})
    #step_dict.update({"risk_free_rate": i})
    step_c = DensityProbability(probability_type, **step_dict) 
    list_of_functions.append(step_c.probability)
x = np.linspace(0.1, 3.0, 100)
for func in list_of_functions:
    plt.plot(x, func(x))
plt.legend(Lista)    

## 5. Options

As explained earlier, the price of a derivative financial product at time $ t $ and $ S(t) $ is given by:

$$
V(t, S(t)) = e^{-r(T-t)} \mathbb{E}_{P_{BS}}[f] = e^{-r(T-t)} \int_0^\infty P_{BS}(S_T; t, S(t)) f(S_T) dS_T \tag{1}
$$

One of the most popular types of derivative products is **options**, which allow the holder to buy (**call option**) or sell (**put option**) the underlying asset at maturity $ T $ for a specified price called the **strike** ($ K $). From a mathematical perspective, the most important aspect of an option is its return, which is typically a *non-linear* function of the underlying asset and the **strike**.

In the *classical_finance* module, returns for several types of **options** have been implemented. For a clear and consistent management of different return options, a Python class called `PayOff` has been implemented in the *payoff_class* module of the *finance* package (**QQuantLib/finance/payoff_class.py**).

To instantiate the class, an input Python dictionary should be provided. The main keys include:

- `pay_off_type*`: This specifies the type of option desired. Currently, the following options are available:
  - `European_Call_Option`
  - `European_Put_Option`
  - `Digital_Call_Option`
  - `Digital_Put_Option`
  - `Futures`

- `strike`: The strike price of the derivative product.

- `coupon`: This is relevant for *Digital_Call_Option* and *Digital_Put_Option*.

The payoffs are obtained from the **QQuantLib/utils/classical_finance** module. Therefore, the keys of the input dictionary should match the keys required for configuring the payoffs in that module.

The class creates the following two properties:

- `pay_off`: A function representing the desired payoff with the appropriate configuration provided by the input dictionary.

- `pay_off_bs`: Provides the exact price of the payoff under the **Black-Scholes** model (the parameter configuration for the **Black-Scholes** model must be included in the input dictionary!).

In [ ]:
from QQuantLib.finance.payoff_class import PayOff

### 5.1 European Call Option

This derivative allows the holder to buy an underlying asset S at a maturity time $T$ at a fixed price $K$ (**strike**) independently of the value of $S_T$. The return of this option at a time $t$ is:

$$f(t, K) = \max(0, S(t)-K)$$

And the final payoff (the return at the maturity time $T$) will be:

$$Payoff(S_T, T, K)= \max(0, S_T-K)=(S_T-K)^+$$


In [ ]:
#dictionary for option configuration
eco_payoff = {
    "pay_off_type": "European_Call_Option",
    "strike": 0.5,
}

eco = PayOff(**eco_payoff)

As can be seen the two properties are python partial functions where the payoff parameters are fixed

In [ ]:
print(type(eco.pay_off))
print(type(eco.pay_off_bs))

The *pay_off* property can compute the expected return for an input asset value!!

In [ ]:
x = np.linspace(0.1, 7.0, 2**9)
plt.plot(x, eco.pay_off(x))

Now we can use the probability density class defined before and the call option pay off for getting the desired price value of our option using the equation $(1)$:

In [ ]:
#This will be BS probability density
bs_pdf = DensityProbability(probability_type, **density_dict)

#Equation (1) for BS and call option pay off
call_option_price = np.sum(bs_pdf.probability(x) * eco.pay_off(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)

print("call_option_price: {}".format(call_option_price))

#### Call option under Black-Scholes

The call option under the **Black-Scholes** model has an analytical solution for computing the *fair* price:

$$V(S_0, K, T)=  e^{rT} \int_0^\infty P_{BS} (S_T; S_0) Payoff(S_T, K, T)dS_T=e^{rT} \int_0^\infty P_{BS} (S_T; S_0)(S_T-K)^+dS_T$$

where

$$P_{BS} (S_T; S_0) = \frac{1}{S_T\sigma\sqrt{2\pi T}}exp\big({-\frac{(\log S_T -\mu)^2}{2\sigma^2 T}}\big)$$

and 
$$\mu = (r-\frac{1}{2}\sigma^2)T+\log S_0$$

The value for a European Call Option under the **Black-Scholes** model has an exact solution:
 
$$V(S(t), t)= S(t)*\Phi(d_1)-Ke^{-r*(T-t)}\Phi(d_2) \tag{2}$$
 
where:

$$d_1 = \frac{log(\frac{S(t)}{K})+(r+\frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}}  \tag{3}$$
$$d_2 = \frac{log(\frac{S(t)}{K})+(r-\frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}} \tag{4}$$
$$\Phi(x)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{x} e^{\frac{-1}{2}\phi^2}d\phi \tag{5}$$


The **pay_off_bs** property allows the user to compute the equation $(2)$ by providing an input Python dictionary with the parameters of the **Black-Scholes** model!!

In [ ]:
print("Analytical price of the Call option under BS model: ", eco.pay_off_bs(**density_dict))

### 5.2 European Put Option

This derivative allows the holder to sell an underlying asset S at a maturity time $T$ at a fixed price $K$ (**strike**) independently of the value of $S_T$. The return of this option at a time $t$ is:

$$f(t, K) = \max(0, K-S(t))$$

And the final payoff (the return at the maturity time $T$) will be:

$$Payoff(S_T, T, K)= \max(0, K-S_T)=(K-S_T)^+$$


In [ ]:
epo_payoff = {
    "pay_off_type": "European_Put_Option",
    "strike": 1.5,
}

epo = PayOff(**epo_payoff)

In [ ]:
x = np.linspace(0.1, 5.5, 2**7)
plt.plot(x, epo.pay_off(x))

In [ ]:
#This will be BS probability density
bs_pdf = DensityProbability(probability_type, **density_dict)

#Equation (1) for BS and put option pay off
put_option_price = np.sum(bs_pdf.probability(x) * epo.pay_off(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)

print("put_option_price: {}".format(put_option_price))

#### Put option under the Black-Scholes Model

The put option has an analytical price when **Black-Scholes** model is used. 

$$V(S(t), t)= Ke^{-r*(T-t)}\Phi(-d_2) - S(t)*\Phi(-d_1) \tag{6}$$

where $d_1$, $d_2$ and $\Phi$ are defined following: $(3)$, $(4)$ and $(5)$ respectively.

The **pay_off_bs** allows the user to compute it by passing the correspondent parameters for the **Black-Scholes**:


In [ ]:
print("Analytical price of the Put option under BS model: ", epo.pay_off_bs(**density_dict))

### 5.3 Digital Call Option

It is a special case of the call option where the return is a step function. It has an analytical price under the **Black-Scholes** model

In [ ]:
dco_payoff = {
    "pay_off_type": "Digital_Call_Option",
    "strike": 0.5,
    "coupon": 1.0
}

dco = PayOff(**dco_payoff)

In [ ]:
x = np.linspace(0.1, 4.0, 2**7)
plt.plot(x, dco.pay_off(x))

In [ ]:
print("Analytical price of the Digital Call Option under  BS model: ", dco.pay_off_bs(**density_dict))

dco_option_price = np.sum(dco.pay_off(x, **density_dict)*bs_pdf.probability(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)
print("Approximate Digital Call Option price: ", dco_option_price)

### 5.3 Digital Put Option

It is a special case of the put option where the return is a step function. It has an analytical price under the **Black-Scholes** model

In [ ]:
dpo_payoff = {
    "pay_off_type": "Digital_Put_Option",
    "strike": 1.5,
    "coupon": 1.0    
}

dpo = PayOff(**dpo_payoff)

In [ ]:
x = np.linspace(0.1, 4.0, 2**7)
plt.plot(x, dpo.pay_off(x))

In [ ]:
print("Analytical price of the Digital Put Option under  BS model: ", dpo.pay_off_bs(**density_dict))

dpo_option_price = np.sum(dpo.pay_off(x, **density_dict)*bs_pdf.probability(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)
print("Approximate Digital Call Option price: ", dpo_option_price)

## 5.4 Futures

The return of a future is a lineal function:

$$Payoff(S_T, T, K)= S_T-K$$

In this case the returns can be **negative**!!!

In [ ]:
future_po_dict = {
    "pay_off_type": "Futures",
    "strike": 1.5,   
}

future = PayOff(**future_po_dict)

In [ ]:
x = np.linspace(0.1, 5.5, 2**9)
plt.plot(x, future.pay_off(x))

In [ ]:
print("Analytical price of the Future under the BS model: ", future.pay_off_bs(**density_dict))

future_price = np.sum(future.pay_off(x, **density_dict)*bs_pdf.probability(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)

print("Approximate Future price : ", future_price)

**BE AWARE** 

In the case of the futures the option price can be **Negative**. This is related with a option that generates losses (it is very important knows when this can happen to avoid it!!)

In [ ]:
future_po_dict = {
    "pay_off_type": "Futures",
    "strike": 2.5,   
}

future = PayOff(**future_po_dict)
print("Analytical price of the Future under the BS model: ", future.pay_off_bs(**density_dict))
future_price = np.sum(future.pay_off(x, **density_dict)*bs_pdf.probability(x)) * np.exp(
    -density_dict['risk_free_rate']*density_dict['maturity']
)

print("Approximate Future price : ", future_price)

## 6. Summary

This notebook has presented several modules for dealing with different derivative options, their payoff and their price estimation. 

As explained in general for getting the *fair* price of the option at a time $t$ is mandatory to compute the expectation value of the distribution probability of the underlying asset at the maturity time of the option multiplied by the payoff of the option (equation $(1)$). 

For computing the integral of this expected value the **Amplitude Estimation** algorithms, developed on the **QQuantLib**, can be used. 

The `AE` and the `Encoding` classes in addition to the `DensityProbability` and the `PayOff` classes (and the *quantum_integration* module) allow the user to build fast implementations for solving option price estimation problems using quantum **AE** techniques!!